In [1]:
#import packages
import pandas as pd
import numpy as np
import csv
import Bio

In [29]:
genes = pd.read_csv('test.txt',sep=' ')

In [30]:
genes

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,FGSG_12766,255.656625,5.810286,2.020458,2.875727,0.004031,0.047326
1,FGSG_00311,151.176689,6.827626,2.487225,2.745078,0.006050,0.048814
2,FGSG_13488,167.295078,8.152338,2.534992,3.215923,0.001300,0.047326


In [31]:
#genes = genes.iloc[:,0]

In [32]:
#genes

In [33]:
df_genes = pd.DataFrame(genes)

In [34]:
df_genes

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,FGSG_12766,255.656625,5.810286,2.020458,2.875727,0.004031,0.047326
1,FGSG_00311,151.176689,6.827626,2.487225,2.745078,0.006050,0.048814
2,FGSG_13488,167.295078,8.152338,2.534992,3.215923,0.001300,0.047326


In [35]:
url_list =[]
id_list = list(df_genes['genes'])
i=0
for row in id_list:
    url_list.append(f'https://www.ncbi.nlm.nih.gov/gene/?term={id_list[i]}')
    i+=1
print('The url-list is completed')

The url-list is completed


In [36]:
url_list

['https://www.ncbi.nlm.nih.gov/gene/?term=FGSG_12766',
 'https://www.ncbi.nlm.nih.gov/gene/?term=FGSG_00311',
 'https://www.ncbi.nlm.nih.gov/gene/?term=FGSG_13488']

In [37]:
#now we need to pack the beautiful soup in a loop and write all protein IDs
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
#url = "https://www.ncbi.nlm.nih.gov/gene/?term=FGSG_09501"  # Replace with the actual URL
ProteinIDs=[]

i=0
print("This will take some time. Drink more coffe while I am working")
for row in url_list:
    try:
        # Fetch the webpage content
        response = requests.get(url_list[i])
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, "html.parser")
    # Find all anchor tags that match the pattern
        for link in soup.find_all("a", href=True):
            if link["href"].startswith("/protein/") and link.text.startswith("XP_"):
                ProteinIDs.append(link.text)
                print(f"ID #{i+1}/{len(url_list)} is {link.text}")
    except:
        print("Failed to retrieve the webpage")
    i+=1
    if i==len(url_list):
        print("My duty is done, MASTER!")
        break

This will take some time. Drink more coffe while I am working
ID #1/3 is XP_011324162.1
ID #2/3 is XP_011315959.1
ID #3/3 is XP_011328828.1
My duty is done, MASTER!


In [38]:
ProteinIDs

['XP_011324162.1', 'XP_011315959.1', 'XP_011328828.1']

In [39]:
df_genes["Protein_ID"] = pd.DataFrame(ProteinIDs)

In [40]:
df_genes

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Protein_ID
0,FGSG_12766,255.656625,5.810286,2.020458,2.875727,0.004031,0.047326,XP_011324162.1
1,FGSG_00311,151.176689,6.827626,2.487225,2.745078,0.006050,0.048814,XP_011315959.1
2,FGSG_13488,167.295078,8.152338,2.534992,3.215923,0.001300,0.047326,XP_011328828.1


In [42]:
ProteinIDs

['XP_011324162.1', 'XP_011315959.1', 'XP_011328828.1']

In [45]:
from Bio.Blast import NCBIWWW
from Bio import Entrez
import time

# ✅ Set your email (required for NCBI API usage)
Entrez.email = "thomas.svoboda@boku.ac.at"  # Replace with your email

# ✅ List of protein IDs (Example)
protein_ids = df_genes["Protein_ID"]  # Add your IDs here
i=0
# ✅ Loop through protein IDs and perform BLASTp
for idx, protein_id in enumerate(protein_ids, 1):
    try:
        print(f"🔍 Running BLASTp for {protein_id} ({idx}/{len(protein_ids)})...")
        
        # ✅ Perform BLASTp search
        result_handle = NCBIWWW.qblast("blastp", "nr", protein_id)

        # ✅ Save results to a file
        output_file = f"blastp_result_{protein_id}.xml"
        with open(output_file, "w") as out_f:
            out_f.write(result_handle.read())
        
        print(f"✅ Results saved: {output_file}")

        # ✅ Wait between queries to avoid NCBI rate limits
        time.sleep(2)

    except Exception as e:
        print(f"❌ Error processing {protein_id}: {e}")

print("🎉 All BLASTp searches completed!")

🔍 Running BLASTp for XP_011324162.1 (1/3)...
✅ Results saved: blastp_result_XP_011324162.1.xml
🔍 Running BLASTp for XP_011315959.1 (2/3)...
✅ Results saved: blastp_result_XP_011315959.1.xml
🔍 Running BLASTp for XP_011328828.1 (3/3)...
✅ Results saved: blastp_result_XP_011328828.1.xml
🎉 All BLASTp searches completed!
